In [1]:
require 'nn';
require 'Attention';
require 'Association';
require 'image';
require 'gnuplot';

# Data Preparation 

In [6]:
--Three Dimensions: 
-- 1=Big/0=Small; 1=Rectangle/0=Triangle; 1=Black/0=White
-- Differentiation between exemplars and category types would be useful
exemplar = torch.Tensor(8,3)
exemplar[1] = torch.Tensor({1,1,1})
exemplar[2] = torch.Tensor({0,1,1})
exemplar[3] = torch.Tensor({1,0,1})
exemplar[4] = torch.Tensor({1,1,0})
exemplar[5] = torch.Tensor({0,0,1})
exemplar[6] = torch.Tensor({1,0,0})
exemplar[7] = torch.Tensor({0,1,0})
exemplar[8] = torch.Tensor({0,0,0})

-- define white exemplars which can hopefully be used as logical indexes
-- See Kruschke 1992, Fig. 4 for this
category = torch.Tensor(6, 8)
category[1] = torch.Tensor({0, 1, 0, 0, 1, 0, 1, 1})
category[2] = torch.Tensor({1, 0, 0, 1, 1, 0, 0, 1})
category[3] = torch.Tensor({1, 0, 0, 0, 1, 0, 1, 1})
category[4] = torch.Tensor({0, 0, 0, 0, 1, 1, 1, 1})
category[5] = torch.Tensor({1, 0, 0, 0, 1, 0, 1, 1})
category[6] = torch.Tensor({0, 1, 1, 1, 0, 0, 0, 1})

alcove = nn.Sequential()
attention = nn.Attention(3, 8, exemplar, 6.5, 1,1)
association = nn.Association(2,8)

alcove:add(attention)
alcove:add(association)


# Training

In [3]:
probabilities = torch.Tensor(50)
target = torch.Tensor(2)
k = 1
-- Over 50 epochs
for i = 1,50 do
    -- For each training exemplar
    for j = 1,8 do
        -- why modulo 8?
        a_out = alcove:forward(exemplar[(j+i)%8 + 1]) -- forward activation
        if exemplar[(j+i)%8 +1][1] == 1  then -- I guess here the discrimination condition is in but I can see it
            target[1] = math.max(1,a_out[1])
            target[2] = math.min(-1,a_out[2])
            if j == 2 then -- Why only if j == 2? The probability for the correct classification given the rule for all stimulus should be recorded
                probabilities[k] = (math.exp(a_out[2]*2.0))/torch.sum(torch.mul(a_out,2.0):exp())
                k = k+1
            end    
        else
            target[1] = math.min(-1,a_out[1])
            target[2] = math.max(1,a_out[2])
            if j == 2 then
                probabilities[k] = (math.exp(a_out[1]*2.0))/torch.sum(torch.mul(a_out,2.0):exp())
                k = k+1
            end
        end
        alcove:backward(exemplar[(j+i)%8 + 1],target-a_out)
    end
    attention:updateParameters(-0.0033)
    association:updateParameters(0.03)
    alcove:zeroGradParameters()
end
x = torch.linspace(1,50,50)


In [4]:
gnuplot.plot(x, probabilities)
print(probabilities)

 0,5000
 0,5362
 0,5727
 0,6078
 0,6408
 0,6717
 0,7013
 0,7301
 0,7581
 0,7850
 0,8106
 0,8344
 0,8564
 0,8764
 0,8945
 0,9107
 0,9250
 0,9374
 0,9483
 0,9575
 0,9654
 0,9720
 0,9776
 0,9822
 0,9859
 0,9890
 0,9914
 0,9934
 0,9950
 0,9962
 0,9971
 0,9979
 0,9984
 0,9988
 0,9992
 0,9994
 0,9996
 0,9997
 0,9998
 0,9999
 0,9999
 0,9999
 1,0000
 1,0000
 1,0000
 1,0000
 1,0000
 1,0000
 1,0000
 1,0000
[torch.DoubleTensor of size 50]

